# Retrieval

In [1]:
from dotenv import load_dotenv
load_dotenv()

from langchain.llms import OpenAI

llm = OpenAI(model='gpt-3.5-turbo-instruct')

C:\Users\annaf\AppData\Local\Temp\ipykernel_7788\1267886439.py:6: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(model='gpt-3.5-turbo-instruct')


In [2]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

caminhos = [
    'assets/Explorando o Universo das IAs com Hugging Face.pdf',
    'assets/Explorando a API da OpenAI.pdf',
    'assets/Explorando a API da OpenAI.pdf'
]

paginas = []

for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())
    
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=['\n\n', '\n', '.', ' ', '']
)

documents = recur_split.split_documents(paginas)

## Modificando metadata

In [3]:
for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('assets/', '')
    doc.metadata['doc_id'] = i

In [4]:
documents[0].metadata

{'source': 'Explorando o Universo das IAs com Hugging Face.pdf',
 'page': 0,
 'doc_id': 0}

In [5]:
# __import__('pysqlite3)
# import sys
# sys.modules['sqlite3] = sys.modules.pop('pysqlite3')

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings_model = OpenAIEmbeddings()

### Criando VectorStore

In [6]:
diretorio = 'docs/chroma_retrieval'

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=diretorio
)

### Semantic Search | Busca Semântica

### Limitações do Semantic Search

#### Trechos com alta similaridade e pouca informação adicional

In [7]:
pergunta = 'O que é a OpenAI?'

docs = vectorstore.similarity_search(pergunta, k=3)

for doc in docs:
    print(f'{doc.page_content}\n')
    print(f'========== {doc.metadata}\n\n')

Explorando a API da OpenAI
01. Bem-vindos ao curso Explorando a API da OpenAI
Olá e bem-vindos ao nosso curso “Explorando a API da OpenAI”! Estou super animado por ter vocês
aqui e mal posso esperar para explorarmos juntos esse universo fascinante da programação e da
inteligência artificial.
Vocês já devem ter percebido que a IA está mudando o jogo em muitos campos, certo? É incrível como
ela está transformando a maneira como lidamos com informações e realiza tarefas que antes pareciam

========== {'doc_id': 283, 'page': 4, 'source': 'Explorando a API da OpenAI.pdf'}


Explorando a API da OpenAI
01. Bem-vindos ao curso Explorando a API da OpenAI
Olá e bem-vindos ao nosso curso “Explorando a API da OpenAI”! Estou super animado por ter vocês
aqui e mal posso esperar para explorarmos juntos esse universo fascinante da programação e da
inteligência artificial.
Vocês já devem ter percebido que a IA está mudando o jogo em muitos campos, certo? É incrível como
ela está transformando a maneira

#### Busca em arquivos específicos não funciona corretamente

In [8]:
pergunta = 'O que a apostila de Hugging Face fala sobre a OpenAI e o ChatGPT??'

docs = vectorstore.similarity_search(pergunta, k=3)

for doc in docs:
    print(f'{doc.page_content}\n')
    print(f'========== {doc.metadata}\n\n')

Explorando o Universo das IAs com Hugging Face
06. Modelos de conversação e geração de texto (chatbots)
Vamos aprender agora a usar modelos de conversação, que funcionam de forma parecida com chatbots
como ChatGPT . Na realidade, estes modelos estão classificados na tarefa de geração de texto (text
generation) no Hugging Face:
Figure 16: Página principal da tarefatext-generation do Hugging Face.
Vamos utilizar o modelo Felladrin/Llama-68M-Chat-v1, disponível no link a seguir:

========== {'doc_id': 86, 'page': 29, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}


Explorando o Universo das IAs com Hugging Face
06. Modelos de conversação e geração de texto (chatbots)
Vamos aprender agora a usar modelos de conversação, que funcionam de forma parecida com chatbots
como ChatGPT . Na realidade, estes modelos estão classificados na tarefa de geração de texto (text
generation) no Hugging Face:
Figure 16: Página principal da tarefatext-generation do Hugging Face.
Vamos utilizar 

### MMR

In [9]:
pergunta = 'O que é a OpenAI?'

docs = vectorstore.max_marginal_relevance_search(pergunta, k=3, fetch_k=10)

for doc in docs:
    print(f'{doc.page_content}\n')
    print(f'========== {doc.metadata}\n\n')

Explorando a API da OpenAI
01. Bem-vindos ao curso Explorando a API da OpenAI
Olá e bem-vindos ao nosso curso “Explorando a API da OpenAI”! Estou super animado por ter vocês
aqui e mal posso esperar para explorarmos juntos esse universo fascinante da programação e da
inteligência artificial.
Vocês já devem ter percebido que a IA está mudando o jogo em muitos campos, certo? É incrível como
ela está transformando a maneira como lidamos com informações e realiza tarefas que antes pareciam

========== {'doc_id': 283, 'page': 4, 'source': 'Explorando a API da OpenAI.pdf'}


Explorando a API da OpenAI
01. Bem-vindos ao curso Explorando a API da OpenAI
Olá e bem-vindos ao nosso curso “Explorando a API da OpenAI”! Estou super animado por ter vocês
aqui e mal posso esperar para explorarmos juntos esse universo fascinante da programação e da
inteligência artificial.
Vocês já devem ter percebido que a IA está mudando o jogo em muitos campos, certo? É incrível como
ela está transformando a maneira

### Filtragem

#### Filtrando para buscar em um arquivo específico

In [10]:
pergunta = 'O que a apostila de Hugging Face fala sobre a OpenAI e o ChatGPT?'

docs = vectorstore.similarity_search(
    pergunta, 
    k=3,
    filter={'source': {'$in': ['Explorando o Universo das IAs com Hugging Face.pdf']}})
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

Explorando o Universo das IAs com Hugging Face
06. Modelos de conversação e geração de texto (chatbots)
Vamos aprender agora a usar modelos de conversação, que funcionam de forma parecida com chatbots
como ChatGPT . Na realidade, estes modelos estão classificados na tarefa de geração de texto (text
generation) no Hugging Face:
Figure 16: Página principal da tarefatext-generation do Hugging Face.
Vamos utilizar o modelo Felladrin/Llama-68M-Chat-v1, disponível no link a seguir:
==========={'doc_id': 86, 'page': 29, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}


Explorando o Universo das IAs com Hugging Face
06. Modelos de conversação e geração de texto (chatbots)
Vamos aprender agora a usar modelos de conversação, que funcionam de forma parecida com chatbots
como ChatGPT . Na realidade, estes modelos estão classificados na tarefa de geração de texto (text
generation) no Hugging Face:
Figure 16: Página principal da tarefatext-generation do Hugging Face.
Vamos utilizar o

#### Filtrando para buscar em uma página específica

In [11]:
pergunta = 'O que a apostila de Hugging Face fala sobre a OpenAI e o ChatGPT?'

docs = vectorstore.similarity_search(
    pergunta, 
    k=3,
    filter={'$and':
            [{'source': {'$in': ['Explorando o Universo das IAs com Hugging Face.pdf']}},
            {'page': {'$in': [10, 11, 12, 13]}}],
            })
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

Explorando o Universo das IAs com Hugging Face
Nas próximas aulas, vamos aprender aos poucos a usar toda a informação contida nesta plataforma!
Asimov Academy 13
==========={'doc_id': 42, 'page': 13, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}


Explorando o Universo das IAs com Hugging Face
Nas próximas aulas, vamos aprender aos poucos a usar toda a informação contida nesta plataforma!
Asimov Academy 13
==========={'doc_id': 42, 'page': 13, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}


Explorando o Universo das IAs com Hugging Face
Nas próximas aulas, vamos aprender aos poucos a usar toda a informação contida nesta plataforma!
Asimov Academy 13
==========={'doc_id': 42, 'page': 13, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}




### LLM Aided Retrieval

In [12]:
from langchain_openai.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.schema import AttributeInfo
import lark

In [13]:
metadata_info = [
    AttributeInfo(
        name='source',
        description='Nome da apostila de onde o texto original foi retirado. Deve ter o valor de: \
            Explorando o Universo das IAs com Hugging Face.pdf ou Explorando a API da OpenAI.pdf',
        type='string'
    ),
    AttributeInfo(
        name='page',
        description='A página da apostila de onde o texto se origina',
        type='integer'
    ),

]

In [15]:
document_description = 'Apostilas de cursos'
llm = OpenAI()
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_description,
    metadata_info,
    verbose=True
)

In [16]:
pergunta = 'O que a apostila de Hugging Face fala sobre a OpenAI e o ChatGPT?'

docs = retriever.get_relevant_documents(pergunta)
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

C:\Users\annaf\AppData\Local\Temp\ipykernel_7788\493495280.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(pergunta)


“famosas” , como o ChatGPT , e construir assim um sistema robusto com diversas IAs para uma variedade
de tarefas. O céu é o limite!
Asimov Academy 88
==========={'doc_id': 265, 'page': 88, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}


“famosas” , como o ChatGPT , e construir assim um sistema robusto com diversas IAs para uma variedade
de tarefas. O céu é o limite!
Asimov Academy 88
==========={'doc_id': 265, 'page': 88, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}


“famosas” , como o ChatGPT , e construir assim um sistema robusto com diversas IAs para uma variedade
de tarefas. O céu é o limite!
Asimov Academy 88
==========={'doc_id': 265, 'page': 88, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}


Explorando o Universo das IAs com Hugging Face
06. Modelos de conversação e geração de texto (chatbots)
Vamos aprender agora a usar modelos de conversação, que funcionam de forma parecida com chatbots
como ChatGPT . Na realidade, estes

In [18]:
pergunta = 'Quais detalhes são descritos na página 44 da apostila Explorando a API da OpenAI?'

docs = retriever.get_relevant_documents(pergunta)
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

Explorando a API da OpenAI
Usos comuns
Alguns casos de uso comuns nos quais o fine-tuning pode aprimorar os resultados incluem:
• Definir o estilo, tom, formato ou outros aspectos qualitativos.
• Aumentar a confiabilidade na geração de uma saída desejada.
• Corrigir falhas ao seguir instruções complexas.
• Lidar com muitos casos atípicos de maneiras específicas.
• Aprender uma nova habilidade ou realizar uma tarefa difícil de articular em uma instrução.
==========={'doc_id': 664, 'page': 44, 'source': 'Explorando a API da OpenAI.pdf'}


Explorando a API da OpenAI
Usos comuns
Alguns casos de uso comuns nos quais o fine-tuning pode aprimorar os resultados incluem:
• Definir o estilo, tom, formato ou outros aspectos qualitativos.
• Aumentar a confiabilidade na geração de uma saída desejada.
• Corrigir falhas ao seguir instruções complexas.
• Lidar com muitos casos atípicos de maneiras específicas.
• Aprender uma nova habilidade ou realizar uma tarefa difícil de articular em uma instrução.